In [1]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

     |████████████████████████████████| 325 kB 18.4 MB/s 
     |████████████████████████████████| 4.0 MB 69.4 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 212 kB 75.8 MB/s 
     |████████████████████████████████| 136 kB 73.4 MB/s 
     |████████████████████████████████| 127 kB 68.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 
     |████████████████████████████████| 895 kB 70.3 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 144 kB 71.0 MB/s 
     |████████████████████████████████| 271 kB 30.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [2]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [3]:
torch.cuda.is_available()
device = torch.device("cuda")

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

Mounted at /content/drive
ajmc  hipe2020	letemps  newseye  sonar  topres19th


In [5]:
train_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/letemps/fr/HIPE-2022-v2.1-letemps-train-fr.tsv'
dev_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/letemps/fr/HIPE-2022-v2.1-letemps-dev-fr.tsv'
test_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/letemps/fr/HIPE-2022-v2.1-letemps-test_allmasked-fr.tsv'

model = 'setu4993/LaBSE'

In [6]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_dev = load_dataset(dev_path)
tsv_test = load_dataset(test_path)

In [7]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [8]:
tsv_train = simple_preprocess(tsv_train)
tsv_dev = simple_preprocess(tsv_dev)
tsv_test = simple_preprocess(tsv_test)
#tsv_train = tsv_train.reset_index()
#tsv_dev.head(100)

In [9]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

ClassLabel(num_classes=8, names=['O', 'B-loc', 'I-loc', 'B-pers', 'I-pers', 'B-org', 'I-org', '_'], id=None)

In [10]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [11]:
train = create_huggingface_file(tsv_train)
val = create_huggingface_file(tsv_dev)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))
print(len(test))

{'id': 0, 'words': ['RÉPUBLIQUE', 'FRANÇAISE', '.'], 'ner': [0, 0, 0]}
{'id': 1, 'words': ['De', 'Bruxelles', ',', 'le', '1}', 'janvier', '.'], 'ner': [0, 1, 0, 0, 0, 0, 0]}
{'id': 2, 'words': ['On', 'mande', 'de', 'Berg', '-', 'op', '-', 'Zoom', 'que', 'le', '17', '.', 'régiment', 'des', 'troupes', 'françaises', 'qui', 'était', 'en', 'garnison', 'dans', 'cette', 'ville', ',', 'vient', 'de', 's', "'", 'embarquer', 'précipitamment', 'pour', 'la', 'Zélande', '.'], 'ner': [0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}
{'id': 3, 'words': ['On', 'y', 'attendait', 'le', '48', '-', 'régiment', 'd', "'", 'infanterie', 'de', 'ligne', 'venant', 'de', 'Hanovre', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}
{'id': 4, 'words': ['—', 'De', 'nombreux', 'transports', 'de', 'chevaux', 'de', 'remonte', ',', 'arrivant', 'de', 'Hanovre', ',', 'ont', 'passé', 'dans', 'nos', 'murs', 'ces', 'jours', 'derniers', ';', 'ils', 'étaient', 

In [12]:
'''LOADERSCRIPT = train_path = '/content/drive/MyDrive/e_ML4NLP2/dataloader.py'
MODEL = "dbmdz/bert-base-german-europeana-cased"'''

'LOADERSCRIPT = train_path = \'/content/drive/MyDrive/e_ML4NLP2/dataloader.py\'\nMODEL = "dbmdz/bert-base-german-europeana-cased"'

In [13]:
'''de_total = load_dataset(LOADERSCRIPT, "de")'''

'de_total = load_dataset(LOADERSCRIPT, "de")'

In [14]:
'''train = de_total["train"]
val = de_total["validation"]
test = de_total["test"]'''

'train = de_total["train"]\nval = de_total["validation"]\ntest = de_total["test"]'

In [15]:
'''tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))
tiny_train = tiny_shuffle_train.select(range(0,80))
tiny_shuffle_test = test.shuffle(seed=42).select(range(100))
tiny_test = tiny_shuffle_test.select(range(80,100))'''

'tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))\ntiny_train = tiny_shuffle_train.select(range(0,80))\ntiny_shuffle_test = test.shuffle(seed=42).select(range(100))\ntiny_test = tiny_shuffle_test.select(range(80,100))'

In [16]:
'''tiny_shuffle_train[10]'''

'tiny_shuffle_train[10]'

In [17]:
'''labels = train.info.features["NE_COARSE_LIT"].feature
labels'''

'labels = train.info.features["NE_COARSE_LIT"].feature\nlabels'

In [18]:
tokenizer = BertTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/4.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

In [19]:
train = train.filter(lambda example, idx: idx != len(train)-1, with_indices=True)
val = val.filter(lambda example, idx: idx != len(val)-1, with_indices=True)
test = test.filter(lambda example, idx: idx != len(test)-1, with_indices=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [20]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/282 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/48 [00:00<?, ?ba/s]

In [21]:
'''# train for 1 epochs, evaluate after each epoch
training_args = TrainingArguments(
    "test_trainer",
    num_train_epochs=3,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
)
trainer.train()'''

'# train for 1 epochs, evaluate after each epoch\ntraining_args = TrainingArguments(\n    "test_trainer",\n    num_train_epochs=3,\n    evaluation_strategy="epoch"\n)\n\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_train,\n    eval_dataset=tokenized_dev,\n)\ntrainer.train()'

In [22]:
training_args = TrainingArguments(
    output_dir = "content/drive/MyDrive/e_ML4NLP2/"
)

In [23]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer.train()

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at setu4993/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, words, id, subwords. If ner, words, id, subwords are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14051
  Num Epochs = 3
  Instantaneous batch size per device = 8
  T

Step,Training Loss
500,0.047100
1000,0.032100
1500,0.030400
2000,0.024100
2500,0.020000
3000,0.020700
3500,0.019100


Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-2000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/pytorch_model.bin
Saving model ch

Step,Training Loss
500,0.047100
1000,0.032100
1500,0.030400
2000,0.024100
2500,0.020000
3000,0.020700
3500,0.019100
4000,0.011800
4500,0.011500
5000,0.010100


Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-4000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-4000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-4500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-4500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-4500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-5000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-5000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-5000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5271, training_loss=0.02205359512840044, metrics={'train_runtime': 1961.9871, 'train_samples_per_second': 21.485, 'train_steps_per_second': 2.687, 'total_flos': 2151374935953600.0, 'train_loss': 0.02205359512840044, 'epoch': 3.0})

In [24]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, words, id, subwords. If ner, words, id, subwords are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2381
  Batch size = 8


In [ ]:
outfile = open("/content/drive/MyDrive/e_ML4NLP2/teamname_bundle4_de_0.tsv", "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [26]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [27]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, words, id, subwords. If ner, words, id, subwords are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1341
  Batch size = 8


(1341, 100, 8)
(1341, 100)


{'eval_accuracy': 0.9850111856823266,
 'eval_f1_macro': 0.5181127608834363,
 'eval_f1_micro': 0.9850111856823266,
 'eval_loss': 0.05100065469741821,
 'eval_runtime': 10.532,
 'eval_samples_per_second': 127.326,
 'eval_steps_per_second': 15.951}